<a href="https://colab.research.google.com/github/YoannRaguenes/Projet-Data-Science-Clubs-De-Foot-Anglais/blob/master/EnglishPlayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#English Player Dataset
The purpose of this code is to create a dataset of almost all professional football players in the English leagues.
We will catch the data from websites:

*   https://sofifa.com
*   https://fbref.com

we are going to use the library Beautifulsoup in order to collect informations on the website.

In [0]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd
from bs4.element import Tag 

###Find the stats of the players
We define our function, the goal of this function is to create a dataframe with some stats about the players.
The shape of the html code is the same for each page, there are tables with the stats for each player in the league.






In [0]:
def getPlayers(Html,div):
  html = urlopen(Html)
  html_soup = BeautifulSoup(html, 'html.parser')
  rows = html_soup.findAll("div",id="all_stats_standard")
  comments =rows[0].findAll(string=lambda text: isinstance(text, Comment))
  for c in comments:
    c.extract()

  sp = BeautifulSoup(c, 'html.parser')
  l = sp.table.tbody.findAll("tr")
  players = []
  for row in range(len(l)):
    if l[row].a!=None :
      player4 = {
                  "name": l[row].a.text,
                  "Nation": l[row].findAll("td",class_="left")[1].text,
                  "Pos": l[row].findAll("td",class_="center")[0].text,
                  "Squad": l[row].findAll("td",class_="left")[2].text,
                 "division": div,
                  "Age": l[row].findAll("td",class_="center")[2].text,
                  "Born": l[row].findAll("td",class_="center")[3].text,
               "MP" : l[row].findAll("td",class_="right")[0].text,
               "Starts" : l[row].findAll("td",class_="right")[1].text,
               "Min" : l[row].findAll("td",class_="right")[2].text,
               "Gls" : l[row].findAll("td",class_="right")[3].text,
               "Ast" : l[row].findAll("td",class_="right")[4].text,
               "PK" : l[row].findAll("td",class_="right")[5].text,
               "PKatt" : l[row].findAll("td",class_="right")[6].text,
               "CrdY" : l[row].findAll("td",class_="right")[7].text,
               "CrdR" : l[row].findAll("td",class_="right")[8].text,
               "Gls/90" : l[row].findAll("td",class_="right")[9].text,
               "Ast/90" : l[row].findAll("td",class_="right")[10].text,
               "G+A" : l[row].findAll("td",class_="right")[11].text,
               "G-PK" : l[row].findAll("td",class_="right")[12].text,
               "G+A-PK" : l[row].findAll("td",class_="right")[13].text,
               'Contract': None,
               "Value": None ,
               'Release_clause':None

              }
      players.append(player4)
  return players


We use the function for the different leagues

In [0]:
#england
df1 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/9/stats/Premier-League-Stats",1))
df2 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/10/stats/Championship-Stats",2))
df3 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/15/stats/League-One-Stats",3))
df4 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/16/stats/League-Two-Stats",4))
df5 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/34/stats/National-League-Stats",5))
tot = pd.concat([df1,df2,df3,df4,df5])
tot[1:-1].to_csv("ENplayers.csv", encoding="utf-8-sig")


In [0]:
tot = pd.concat([df1,df2,df3,df4,df5])

Read the csv file 

In [0]:
tot[1:-1].to_csv("ENplayers.csv", encoding="utf-8-sig")

In [21]:
df = pd.read_csv('ENplayers.csv')
df

,Unnamed: 0,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Contract,Value,Release_clause
0,1,Max Aarons,eng ENG,DF,Norwich City,1,19.0,2000.0,27,27,"2,430",0,1.0,0,0,6,0,0.00,0.04,0.04,0.00,0.04,NaN,NaN,NaN
1,2,Tammy Abraham,eng ENG,FW,Chelsea,1,21.0,1997.0,25,23,"1,945",13,3.0,0,0,2,0,0.60,0.14,0.74,0.60,0.74,NaN,NaN,NaN
2,3,Che Adams,eng ENG,FW,Southampton,1,23.0,1996.0,22,8,725,0,2.0,0,0,0,0,0.00,0.25,0.25,0.00,0.25,NaN,NaN,NaN
3,4,Adrián,es ESP,GK,Liverpool,1,32.0,1987.0,11,9,873,0,0.0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
4,5,Sergio Agüero,ar ARG,FW,Manchester City,1,31.0,1988.0,22,17,"1,401",16,3.0,2,3,1,0,1.03,0.19,1.22,0.90,1.09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3202,670,Mark Yeates,ie IRL,"FW,MF",Fylde,5,34.0,1985.0,29,18,"1,685",1,NaN,0,0,3,0,0.05,NaN,NaN,0.05,NaN,NaN,NaN,NaN
3203,671,Luke Young,eng ENG,"MF,DF",Wrexham,5,26.0,1993.0,37,36,"3,244",3,NaN,0,0,5,0,0.08,NaN,NaN,0.08,NaN,NaN,NaN,NaN
3204,672,Adi Yussuf,tz TAN,"FW,MF",Boreham Wood,5,27.0,1992.0,3,0,23,0,NaN,0,0,0,0,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN
3205,673,Adi Yussuf,tz TAN,FW,Solihull Moors,5,27.0,1992.0,9,3,304,0,NaN,0,0,0,0,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN


Now we can do some requests and analysis with this dataframe.


In [9]:
df.loc[ (df['Age']<=20) & (df['Gls']>=10)]


,Unnamed: 0,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Contract,Value,Release_clause
880,389,Bryan Mbeumo,fr FRA,FW,Brentford,2,19.0,1999.0,34,29,"2,404",14,6.0,0,0,3,0,0.52,0.22,0.75,0.52,0.75,NaN,NaN,NaN
1375,206,Marcus Forss,fi FIN,FW,AFC Wimbledon,3,20.0,1999.0,18,17,"1,346",11,1.0,3,3,3,1,0.74,0.07,0.80,0.53,0.60,NaN,NaN,NaN


In [0]:
r1= df.sort_values(by=["Gls"]).tail(10)

In [11]:
df.loc[(df['Pos']== 'MF')&(df['Ast']>=10) & (df['Gls']>=5)].sort_values(by=["G+A"])

,Unnamed: 0,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Contract,Value,Release_clause
1092,601,John Swift,eng ENG,MF,Reading,2,24.0,1995.0,32,31,"2,652",5,10.0,0,0,6,1,0.17,0.34,0.51,0.17,0.51,NaN,NaN,NaN
2511,682,Randell Williams,eng ENG,MF,Exeter City,4,22.0,1996.0,37,33,"3,005",5,13.0,0,0,4,0,0.15,0.39,0.54,0.15,0.54,NaN,NaN,NaN
1378,209,Scott Fraser,sco SCO,MF,Burton Albion,3,24.0,1995.0,30,25,"2,296",5,11.0,0,0,2,0,0.20,0.43,0.63,0.20,0.63,NaN,NaN,NaN
100,101,Kevin De Bruyne,be BEL,MF,Manchester City,1,28.0,1991.0,26,25,"2,156",8,17.0,0,0,2,0,0.33,0.71,1.04,0.33,1.04,NaN,NaN,NaN


### Find the market value of the players 
We are going to use the web site:   https://sofifa.com

In [0]:
from urllib.request import urlopen, Request

We collect statistics and market values from players thanks to this function

In [0]:
def get_data(Url):
  headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
  reg_url = Url
  req = Request(url=reg_url, headers=headers) 
  html2 = urlopen(req)
  html_soup = BeautifulSoup(html2, 'html.parser')
  rows = html_soup.findAll("div",class_='column col-auto')
  rows[0].findAll('td',class_="col-name")[0].text
  trs =rows[0].table.findAll('tr')
  cells = html_soup.findAll("tr")

  liste=[]
  for i in range(1,len(trs)):
    try:
      player4 = {
                  "name": trs[i].a['data-tooltip'],
                "age": trs[i].findAll('td',class_='col-ae')[0].text,
                  "Overall_rating": cells[i].findAll('td')[3].text,
                 "Potential": cells[i].findAll('td')[4].text,
                  "Team": cells[i].findAll('td')[5].a.text,
                  "Contract": cells[i].findAll('td')[5].div.div.text[1:],
                  "Best_position": cells[i].findAll('td')[6].text,
                  "Growth": cells[i].findAll('td')[7].text,
                  "Value": cells[i].findAll('td')[8].text,
                  "Release_clause": cells[i].findAll('td')[9].text,
                  "PAC": cells[i].findAll('td')[10].text,
                  "SHO": cells[i].findAll('td')[11].text,
                  "PAS": cells[i].findAll('td')[12].text,
                  "DRI": cells[i].findAll('td')[13].text,
                  "DEF": cells[i].findAll('td')[14].text,
                  "PHY": cells[i].findAll('td')[15].text,
                  "Lien_photo": cells[i].findAll('td')[0].figure.img['data-src']
                  
                 }
      liste.append(player4)
      
    except:
      print()
      
  return liste



In [0]:
url = 'https://sofifa.com/players?type=all&lg%5B0%5D=13&lg%5B1%5D=14&lg%5B2%5D=60&lg%5B3%5D=61&col=gu&sort=desc&showCol%5B%5D=ae&showCol%5B%5D=oa&showCol%5B%5D=pt&showCol%5B%5D=bp&showCol%5B%5D=gu&showCol%5B%5D=vl&showCol%5B%5D=rc&showCol%5B%5D=pac&showCol%5B%5D=sho&showCol%5B%5D=pas&showCol%5B%5D=dri&showCol%5B%5D=def&showCol%5B%5D=phy&offset='
urlListe =[]
i=0
while i<=2660 :
  url1 = url + str(i)
  urlListe.append(url1)
  url1=url
  i = i + 60


In [0]:
dtListe = []
for l in urlListe:
  dtListe.append(pd.DataFrame(get_data(l)))

we can create a dataset with a lot of statistics about the players and their market value

In [16]:
dfval = pd.concat(dtListe)
dfval.to_csv("playerVal.csv", encoding="utf-8-sig")
dfval = pd.read_csv('playerVal.csv')
dfval

,Unnamed: 0,name,age,Overall_rating,Potential,Team,Contract,Best_position,Growth,Value,Release_clause,PAC,SHO,PAS,DRI,DEF,PHY,Lien_photo
0,0,Ricky-Jade Jones,16,59,84,Peterborough United,2019 ~ 2022,ST,25,€350K,€963K,92,56,50,58,27,56,https://cdn.sofifa.com/players/254/482/20_60.png
1,1,Nelson Agho,16,52,76,Port Vale,2018 ~ 2023,ST,24,€110K,€303K,76,48,34,47,17,67,https://cdn.sofifa.com/players/251/478/20_60.png
2,2,Armando Broja,17,61,84,Chelsea,2020 ~ 2022,ST,23,€550K,€1.6M,71,59,45,58,28,63,https://cdn.sofifa.com/players/256/268/20_60.png
3,3,Harvey Elliott,16,64,87,Liverpool,2019 ~ 2023,CAM,23,€975K,€2.8M,75,53,58,69,30,37,https://cdn.sofifa.com/players/246/174/20_60.png
4,4,Taylor Harwood-Bellis,17,60,83,Manchester City,2019 ~ 2021,CB,23,€425K,€1.2M,63,33,46,51,60,61,https://cdn.sofifa.com/players/252/793/20_60.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,34,Nathan Pond,34,61,61,Salford City,2018 ~ 2020,CB,0,€90K,€158K,30,32,42,39,60,72,https://cdn.sofifa.com/players/209/277/20_60.png
2395,35,Chris Neal,33,61,61,Salford City,2018 ~ 2020,GK,0,€140K,€245K,61,60,58,60,57,61,https://cdn.sofifa.com/players/166/117/20_60.png
2396,36,Tom Elliott,28,67,67,Salford City,2020 ~ 2022,ST,0,€800K,€1.4M,77,61,42,64,30,79,https://cdn.sofifa.com/players/183/425/20_60.png
2397,37,Jake Jervis,27,64,64,Salford City,"Jun 30, 2020 On Loan",RM,0,€475K,€0,83,59,54,62,34,72,https://cdn.sofifa.com/players/198/269/20_60.png


This step is very long, this goal is to add the market value  in the ENplayers dataset.

In [0]:
for i in range(len(dfval.name)-1):
  for j in range(len(df.name) -1):
    if pd.Series(dfval['name'][i]).str.contains(df['name'][j])[0]:
      df.loc[(df['name']==df['name'][j]), 'Value']= dfval['Value'][i]
      df.loc[(df['name']==df['name'][j]), 'Contract']= dfval['Contract'][i]
      df.loc[(df['name']==df['name'][j]), 'Release_clause']= dfval['Release_clause'][i]
      print(dfval['name'][i])
      print(i/(len(dfval.name))*100 )

In [0]:
df

In [0]:
df.to_csv('player.csv',encoding="utf-8-sig")